In [1]:
import os
import shutil
import json
import glob
import csv
import threading

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp 
import networkx as nx 

from manager  import RunDict, ResultDict, ItemDict
from analysis import utils
from typing   import List, Dict

%load_ext autoreload
%autoreload 2

In [2]:

INFRA   = "gcp" 

# MINOR
PREFIX  = "2024-07-21-13-08-04"

# MAJOR
PREFIX  = "2024-08-02-13-58-29"

# LEMON
PREFIX  = "2024-08-04-13-01-35"

DIRPATH = f"analysis/data/treefinder-{INFRA}-{PREFIX}"
DIRPATH = os.path.join(os.getcwd(), DIRPATH)

if not os.path.isdir(DIRPATH):
    raise RuntimeError(f"Not a directory: {DIRPATH}")
    
DIRPATH

'/home/duclos/Documents/work/master-arbeit/analysis/data/treefinder-gcp-2024-08-04-13-01-35'

In [3]:
from analysis import experiment
LEMON = experiment.Experiment(DIRPATH)

PROC[0]
PROC[2]
PROC[1]
PROC[3]
PARSED RESULT[xrYzfLnychxnLOUvULUt] from RUN[LEMON-A-STRESS-2-NONE-W_1]
PARSED RESULT[YZjqlUwPTwKtjOiqEBGg] from RUN[LEMON-A-1-NONE-W_1]
PARSED RESULT[vGHVYxiVmGXSEgotNHZo] from RUN[LEMON-B-STRESS-3-NONE-W_1]
PARSED RESULT[qrHrHaxxbddtIsJFiWeC] from RUN[LEMON-B-3-NONE-W_1]
PARSED RESULT[kcrqqwLDXXVjBaFCEHcZ] from RUN[LEMON-A-STRESS-3-NONE-W_1]
PARSED RESULT[ZvhwJPNZSDhKMnYojjjp] from RUN[LEMON-A-2-NONE-W_1]
PARSED RESULT[REaaIjZbFyQnMGeePuah] from RUN[LEMON-C-STRESS-1-NONE-W_1]
PARSED RESULT[IEpWbtXKzYuPOiATUltl] from RUN[LEMON-C-1-NONE-W_1]
PARSED RESULT[SKbZJCmWTpmKauLezhBm] from RUN[LEMON-B-STRESS-1-NONE-W_1]
PARSED RESULT[YjdlaWZgfApWSkSCojdN] from RUN[LEMON-A-3-NONE-W_1]
PARSED RESULT[qoXNuSmiYGCklhBMXcjl] from RUN[LEMON-C-STRESS-2-NONE-W_1]
PARSED RESULT[yupEbghVGUgDrsityOwn] from RUN[LEMON-C-2-NONE-W_1]
PARSED RESULT[itEEesLhBWPymBLXfYtv] from RUN[LEMON-B-STRESS-2-NONE-W_1]
PARSED RESULT[LaEKVyLgwzNElLQCPRQw] from RUN[LEMON-B-1-NONE-W_1]
PROC[2]: 

In [3]:
from analysis import experiment
EXPERIMENT = experiment.Experiment(DIRPATH)

PROC[0]
PROC[1]
PROC[2]
PROC[3]
PARSED RESULT[mkpoGMqvTGDUTdozyPkI] from RUN[BEST-p90-W_5]
PARSED RESULT[oMvGQbjcNPTXZtCCNMnQ] from RUN[BEST-REBUILD-1-p50-W_5]
PARSED RESULT[XcfOFtNfLbbhpUMYlRDT] from RUN[BEST-REBUILD-2-heuristic-W_5]
PARSED RESULT[KCogBJCyuAPEHoYAcGAY] from RUN[BEST-p90-W_5]
PARSED RESULT[NwLHZmPETxoslpAscYxF] from RUN[BEST-p90-W_5]
PARSED RESULT[LaepODJCUNiTMoEzySkC] from RUN[BEST-p90-W_5]
PARSED RESULT[RLqfFLubegaZyyNlkJQi] from RUN[BEST-p90-W_5]
PARSED RESULT[brALPYrVUgNHZPGVTxjd] from RUN[BEST-p90-W_5]
PARSED RESULT[JiAufwqTPcaNVfWavnRF] from RUN[RAND-NONE-W_5]
PARSED RESULT[LYwUWGlShXSjNPNVUQnb] from RUN[BEST-p90-W_5]
PARSED RESULT[eIZBhQCSrhXXiqTnbpkN] from RUN[BEST-REBUILD-1-p50-W_5]
PARSED RESULT[eQnbhudnFbPTFouBcsjj] from RUN[BEST-REBUILD-2-heuristic-W_5]
PARSED RESULT[abPeavyqJnpKvzEYGnJS] from RUN[BEST-p90-W_5]
PARSED RESULT[FaWKtFYDZOMMZrDTaRzD] from RUN[RAND-NONE-W_5]
PARSED RESULT[gYlEuuBJkOzirsnUDPis] from RUN[BEST-REBUILD-1-p50-W_5]
PARSED RESULT[zqsAd

In [ ]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "plot")
utils.mkdir(dir)
    
core.plotAll(dir)

In [ ]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir)
    
core.plotTarget(name="BEST", key="heuristic", dir=dir)

In [5]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir)


file="BEST-HeuristicxW5xRANDxWORST.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_5" },
     {"key": "p90",       "name": "WORST",          "root": "W_5" },
     {"key": "NONE",       "name": "RAND",           "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x WORST x RAND", 
                        file=file) 

file="BEST-HeuristicxW5xLEMONS.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_5" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
] 
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x LEMONS", 
                        file=file) 

    
file="BEST-HeuristicxW5xOthers.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "p90",       "name": "BEST",            "root": "W_5" },
    {"key": "p50",       "name": "BEST",            "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x OTHERS", 
                        file=file) 

file="BEST-HeuristicxW3xOthers.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "p90",       "name": "BEST",            "root": "W_3" },
    {"key": "p50",       "name": "BEST",            "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x OTHERS", 
                        file=file) 

file="BEST-HeuristicxW3xLEMONS.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
] 
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x LEMONS", 
                        file=file) 

file="BEST-HeuristicxW5xREBUILDS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x REBUILDS", 
                        file=file) 

file="BEST-HeuristicxW3xREBUILDS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x REBUILDS", 
                        file=file) 



PLOTTED: BEST-HeuristicxW5xRANDxWORST.png
PLOTTED: BEST-HeuristicxW5xLEMONS.png
PLOTTED: BEST-HeuristicxW5xOthers.png
PLOTTED: BEST-HeuristicxW3xOthers.png
PLOTTED: BEST-HeuristicxW3xLEMONS.png
PLOTTED: BEST-HeuristicxW5xREBUILDS.png
PLOTTED: BEST-HeuristicxW3xREBUILDS.png


In [6]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir) 
    
# dir = "/home/duclos/Documents/work/master-arbeit/docs/slidev/plot"

d = {"key": "heuristic", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0] 

file="BEST-HeuristicxW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 1-3", 
                        file=file) 

file="BEST-HeuristicxW5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 4-7", 
                        file=file) 

file="BEST-HeuristicxW5xSTAGESALL.png"
# core.plotStageComparison(run=run, 
#                         stages=run["stages"], 
#                         rdir=dir, 
#                         title="BEST HEURISTIC: STAGES ALL", 
#                         file=file) 

d = {"key": "p90", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]  

file="BEST-p90xW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST p90: STAGES 1-3", 
                        file=file) 

file="BEST-p90W5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST p90: STAGES 4-7", 
                        file=file) 

d = {"key": "p50", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]  

file="BEST-p50xW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST p50: STAGES 1-3", 
                        file=file) 

file="BEST-p50W5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST p50: STAGES 4-7", 
                        file=file)

PLOTTED: BEST-HeuristicxW5xSTAGES1-3.png
PLOTTED: BEST-HeuristicxW5xSTAGES4-7.png
PLOTTED: BEST-p90xW5xSTAGES1-3.png
PLOTTED: BEST-p90W5xSTAGES4-7.png
PLOTTED: BEST-p50xW5xSTAGES1-3.png
PLOTTED: BEST-p50W5xSTAGES4-7.png


In [6]:
from analysis import core, args

core.EXPERIMENT = LEMON
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir) 
    
dir = "/home/duclos/Documents/work/master-arbeit/docs/slidev/plot" 

file="LEMON-AxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-A-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-A-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-A RUNS", file=file) 

file="LEMON-BxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-B RUNS", file=file) 

file="LEMON-CxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-C RUNS", file=file)

file="LEMONxOthers.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON PARAMETRIZATION", file=file) 

file="LEMON-STRESSxOthers.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-STRESS-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-STRESS-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-STRESS-1",            "root": "" },
] 

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON STRESS PARAMETRIZATION", file=file) 

PLOTTED: LEMON-AxRUNS.png
PLOTTED: LEMON-BxRUNS.png
PLOTTED: LEMON-CxRUNS.png
PLOTTED: LEMONxOthers.png
PLOTTED: LEMON-STRESSxOthers.png
